In [1]:
import time
import json
import pandas as pd
from py2neo import Graph, Node, Relationship

In [10]:
graph = Graph("bolt://neo4j-allquanta:7687", auth=('neo4j','myneo'))

print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 278,590,931 nodes and 1,844,902,937 relationships!


## Citations

In [4]:
venue = "" # """(q.venue="CA: A Cancer Journal for Clinicians" OR q.venue="The New England Journal of Medicine" OR q.venue="The Lancet" OR q.venue="Chemical Reviews" OR q.venue="Nature Reviews Drug Discovery" OR q.venue="JAMA" OR q.venue="Nature Reviews Cancer" OR q.venue="Nature Reviews Immunology" OR q.venue="Nature" OR q.venue="Nature Reviews Genetics" OR q.venue="Science" OR q.venue="Chemical Society Reviews" OR q.venue="Nature Materials" OR q.venue="Nature Nanotechnology" OR q.venue="Lancet Oncology" OR q.venue="Reviews of Modern Physics" OR q.venue="Nature Biotechnology" OR q.venue="Nature Reviews Molecular Cell Biology" OR q.venue="Nature Reviews Neuroscience" OR q.venue="Nature Medicine" OR q.venue="Nature Photonics" OR q.venue="Nature Reviews Microbiology" OR q.venue="Cell" OR q.venue="Advances in Physics" OR q.venue="Energy and Environmental Science" OR q.venue="World Psychiatry") AND """
min_total_citations = 100
starting_year = 2003
ending_year = 2008
n_years = 5
cite_multiple = 2
min_multiple_count = 4
cite_count_string = "\t"+",\n\t".join(["SIZE((q)<-[:CITES]-(:Quanta {{year: q.year+{}}})) as c{}".format(i,i) for i in range(1,n_years+1)])
cite_multiple_string = "\t"+",\n\t".join(["CASE WHEN (c{} > {}*c{}) THEN 1 ELSE 0 END as b{}".format(i+1,cite_multiple,i,i) for i in range(1,n_years)])
c_string = ", ".join(["c{}".format(i) for i in range(1,n_years+1)])
b_string = "+".join(["b{}".format(i) for i in range(1,n_years)])

query = """
MATCH (q:Quanta) 
WHERE EXISTS(q.fos) AND (q.n_citation>{}) AND (q.year>={} AND q.year <= {}) {}
WITH q.title as title, q.venue as venue, q.year as year, q.n_citation as n_citation, q.pageRank_2018 as pagerank,
    {}
WITH *,
    {}
WHERE ({})>={}
RETURN title, venue, year, n_citation, {}, pagerank, {} as sum_bool
LIMIT 50
""".format(min_total_citations, starting_year, ending_year, venue, cite_count_string, cite_multiple_string, b_string, min_multiple_count, c_string, b_string)

print(query)
query_start_time = time.time()
df = graph.run(query).to_data_frame()
query_end_time = time.time()
print("Done in ({:.2f} minutes).".format((query_end_time-query_start_time)/60))


MATCH (q:Quanta) 
WHERE EXISTS(q.fos) AND (q.n_citation>100) AND (q.year>=2003 AND q.year <= 2008) 
WITH q.title as title, q.venue as venue, q.year as year, q.n_citation as n_citation, q.pageRank_2018 as pagerank,
    	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+1})) as c1,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+2})) as c2,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+3})) as c3,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+4})) as c4,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+5})) as c5
WITH *,
    	CASE WHEN (c2 > 2*c1) THEN 1 ELSE 0 END as b1,
	CASE WHEN (c3 > 2*c2) THEN 1 ELSE 0 END as b2,
	CASE WHEN (c4 > 2*c3) THEN 1 ELSE 0 END as b3,
	CASE WHEN (c5 > 2*c4) THEN 1 ELSE 0 END as b4
WHERE (b1+b2+b3+b4)>=4
RETURN title, venue, year, n_citation, c1, c2, c3, c4, c5, pagerank, b1+b2+b3+b4 as sum_bool
LIMIT 50



KeyboardInterrupt: 

Exception ignored in: 'neo4j.bolt._io.ChunkedInputBuffer.receive'
KeyboardInterrupt


ServiceUnavailable: Failed to read from closed connection Address(host='172.22.0.3', port=7687)

In [ ]:
df

In [13]:
# Get citation counts for papers of interest
n_years = 6
cite_count_string = "\t"+",\n\t".join(["SIZE((q)<-[:CITES]-(:Quanta {{year: q.year+{}}})) as c{}".format(i,i) for i in range(1,n_years+1)])
c_string = ", ".join(["c{}".format(i) for i in range(1,n_years+1)])

title = "CRISPR Provides Acquired Resistance Against Viruses in Prokaryotes"
query = """
MATCH (q:Quanta {{title: "{}"}}) 
WITH q.title as title, q.venue as venue, q.year as year, q.n_citation as n_citation, q.pageRank_2018 as pagerank,
{}  
RETURN title, n_citation, {}, pagerank
""".format(title, cite_count_string, c_string)
# print(query)
df = graph.run(query).to_data_frame()
df.head()

,c1,c2,c3,c4,c5,c6,n_citation,pagerank,title
0,58,82,89,92,133,192,2175,12.471728,CRISPR Provides Acquired Resistance Against Vi...


In [6]:
# Get average number of citaitons for Nature and Science
n_years = 11
cite_count_string = "\t"+",\n\t".join(["SIZE((q)<-[:CITES]-(:Quanta {{year: q.year+{}}})) as c{}".format(i,i) for i in range(1,n_years+1)])
c_string = ", ".join(["c{}".format(i) for i in range(1,n_years+1)])
avg_c_string = ", ".join(["avg(c{})".format(i) for i in range(1,n_years+1)])
std_c_string = ", ".join(["stdev(c{})".format(i) for i in range(1,n_years+1)])

query = """
MATCH (q:Quanta) 
WHERE (q.venue="Nature" OR q.venue="Science") AND (q.year <= 2016 AND q.year >=2007)
WITH q.year as year, q.n_citation as n_citation, q.pageRank_2018 as pagerank,
{}
RETURN avg(year), avg(n_citation), avg(pagerank), {}, {}
""".format(cite_count_string, avg_c_string, std_c_string)
# print(query)
df = graph.run(query).to_data_frame()
df.head()

,avg(c1),avg(c10),avg(c11),avg(c2),avg(c3),avg(c4),avg(c5),avg(c6),avg(c7),avg(c8),...,stdev(c10),stdev(c11),stdev(c2),stdev(c3),stdev(c4),stdev(c5),stdev(c6),stdev(c7),stdev(c8),stdev(c9)
0,8.088031,0.418779,0.001035,8.968721,8.176588,7.128462,5.898392,4.616145,3.401602,2.278571,...,4.700689,0.04116,27.956862,28.811901,27.441111,24.639662,22.101952,19.438173,15.737673,10.779695


In [8]:
df.T

,0
avg(c1),8.088031
avg(c10),0.418779
avg(c11),0.001035
avg(c2),8.968721
avg(c3),8.176588
avg(c4),7.128462
avg(c5),5.898392
avg(c6),4.616145
avg(c7),3.401602
avg(c8),2.278571
